In [ ]:
# Run to start driver
# Note that chromedriver.exe must be the same version as your Chrome browser

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

url = "https://scaledagile.com/business-solutions/find-a-partner/"
driver = Chrome()
driver.get(url)
data = []

In [ ]:
# Counts how many companies in a page
companies = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.XPATH, "//a[@class='partner-card']")))

# Loops for every company in a page
for i in range(len(companies)):
    companies = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.XPATH, "//a[@class='partner-card']")))

    company = companies[i]
    company.click()

    #Scrape Info
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    try: # Skip if site didnt load
        name = soup.find("h1", class_="summary-title").text
    except AttributeError:
        driver.back()
        continue

    try: # Leave blank if no email
        email = soup.find("lightning-formatted-email").text[6:]
    except AttributeError:
        email = ""
    
    try:  # Leave blank if no phone number
        phone = soup.find("lightning-formatted-phone").text
    except AttributeError:
        phone = ""
    
    # Store dict in list
    data.append({"name": name, "email": email, "phone": phone})
    print(i+1, name, email, phone)

    driver.back()

# Auto next page not yet done, manually press next page in driver
# and run this code block again
print("Go to next page")  

In [ ]:
# Write data on csv file
import csv

fields = ['Name', 'Email', 'Phone']

keys = data[0].keys()

with open('all_companies.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

In [ ]:
driver.quit()